In [1]:
import pyam
import math
import numpy as np
import os
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re
from typing import Dict, List, Optional

<IPython.core.display.Javascript object>

### Define path of directory to write out plots

In [2]:
path_dic = '/Users/haukeh/Library/CloudStorage/OneDrive-KTH/box_files/KTHwork/paper/PhD3-Diagnostics on OSeMBE/plots/240122'

In [3]:
wp1_models = [
    'IMAGE 3.2',
    'MESSAGEix-GLOBIOM 1.2', 
    'PROMETHEUS 1.2',
    'REMIND 2.1',  
    'TIAM-ECN 1.2',  
    'WITCH 5.1',
    'Euro-Calliope 2.0',
    'LIMES 2.38',
    'MEESA v1.2',
    'OSeMBE v1.0.0', 
    'PRIMES 2022'
    ]
license_status = {
    'open-source': ['MESSAGEix-GLOBIOM 1.2', 'WITCH 5.1', 'Euro-Calliope 2.0', 'OSeMBE v1.0.0'],
    'proprietary': ['IMAGE 3.2', 'PROMETHEUS 1.2', 'REMIND 2.1', 'TIAM-ECN 1.2', 'LIMES 2.38', 'MEESA v1.2', 'PRIMES 2022']
}
model_type = {
    'IAM': ['IMAGE 3.2', 'MESSAGEix-GLOBIOM 1.2', 'PROMETHEUS 1.2', 'REMIND 2.1', 'TIAM-ECN 1.2',  'WITCH 5.1' ],
    'ESM': ['Euro-Calliope 2.0', 'OSeMBE v1.0.0', 'LIMES 2.38', 'MEESA v1.2', 'PRIMES 2022']
}
diag_scenarios = [
    'DIAG-Base', 
    'DIAG-NPI',
    'DIAG-C400-lin',
    'DIAG-C80-gr5',
    'DIAG-C0to80-gr5',
    'DIAG-NZero',
    'DIAG-C400-lin-LimBio',
    'DIAG-C400-lin-LimCCS',
    'DIAG-C400-lin-LimNuclear',
    'DIAG-C400-lin-HighVRE',
    'DIAG-C400-lin-HighElectrification',
    'DIAG-C400-lin-HighH2',
    'DIAG-C400-lin-ResidualFossil',
    'DIAG-C400-lin-HighEff'
    ]
interest_scenarios = [
    'DIAG-C400-lin'
]
common_variables = [
    'Capacity|Electricity|Nuclear',
    'Capacity|Electricity|Solar|CSP',
    'Capacity|Electricity|Solar|PV',
    'Capacity|Electricity|Solar|PV|Rooftop',
    'Capacity|Electricity|Solar|PV|Utility',
    'Capacity|Electricity|Wind',
    'Capacity|Electricity|Wind|Offshore',
    'Capacity|Electricity|Wind|Onshore',
    'Capital Cost|Electricity|Nuclear',
    'Capital Cost|Electricity|Solar|CSP',
    'Capital Cost|Electricity|Solar|PV',
    'Capital Cost|Electricity|Solar|PV|Utility',
    'Capital Cost|Electricity|Wind|Offshore',
    'Capital Cost|Electricity|Wind|Onshore',
    'Capital Cost|Hydrogen|Electricity',
    'Efficiency|Hydrogen|Electricity',
    'Emissions|CO2|Energy|Supply|Electricity',
    'Emissions|CH4|Energy|Supply',
    'Final Energy',
    'Final Energy|Electricity',
    'Final Energy|Commercial',
    'Final Energy|Commercial|Electricity',
    'Final Energy|Heat',
    'Final Energy|Residential',
    'Final Energy|Residential|Electricity',
    'Final Energy|Residential and Commercial',
    'Final Energy|Residential and Commercial|Electricity',
    'Final Energy|Transportation',
    'Final Energy|Transportation|Electricity',
    'Lifetime|Electricity|Nuclear',
    'Primary Energy|Biomass|Electricity',
    'Primary Energy|Coal|Electricity',
    'Primary Energy|Gas|Electricity',
    'Primary Energy|Nuclear',
    'Primary Energy|Oil|Electricity',
    'Primary Energy|Solar',
    'Primary Energy|Wind|Offshore',
    'Primary Energy|Wind|Onshore',
    'Secondary Energy|Electricity',
    'Secondary Energy|Electricity|Biomass',
    'Secondary Energy|Electricity|Coal',
    'Secondary Energy|Electricity|Gas',
    'Secondary Energy|Electricity|Geothermal',
    'Secondary Energy|Electricity|Hydro',
    'Secondary Energy|Electricity|Nuclear',
    'Secondary Energy|Electricity|Ocean',
    'Secondary Energy|Electricity|Oil',
    'Secondary Energy|Electricity|Solar',
    'Secondary Energy|Electricity|Solar|PV',
    'Secondary Energy|Electricity|Solar|CSP',
    'Secondary Energy|Electricity|Wind',
    'Secondary Energy|Electricity|Wind|Offshore',
    'Secondary Energy|Electricity|Wind|Onshore',
    'Secondary Energy|Heat',
    'Secondary Energy|Hydrogen',
    'Secondary Energy|Hydrogen|Electricity',
]

In [4]:
# Mapping by model regions
dic_model_reg = {
    'Two Regions': {
        'Western Europe': {
            'Euro-Calliope 2.0': ['Austria', 'Belgium', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Ireland', 'Italy', 'Luxembourg', 'Netherlands', 'Portugal', 'Spain', 'Sweden', 'United Kingdom'],
            'IMAGE 3.2': ['WEU'],
            'LIMES 2.38': ['Austria', 'Belgium', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Ireland', 'Italy', 'Luxembourg', 'Malta', 'Netherlands', 'Portugal', 'Spain', 'Sweden', 'United Kingdom'],
            'MEESA v1.2': ['ENC', 'ESC', 'ESW', 'EWN', 'UKI', 'DEU', 'FRA'],
            'MESSAGEix-GLOBIOM 1.2': ['WEU'],
            'OSeMBE v1.0.0': ['AUT', 'BEL', 'DEU', 'DNK', 'ESP', 'FIN', 'FRA', 'GBR', 'GRC', 'IRL', 'ITA', 'LUX', 'MLT', 'NLD', 'PRT', 'SWE'],
            'PRIMES 2022': ['Scandinavia (IP)', 'Southern Europe (IP)', 'Iberian Peninsula (IP)', 'Central Europe (IP)', 'United Kingdom & Ireland (IP)'],
            'PROMETHEUS 1.2': ['Western Europe'],
            'REMIND 2.1':['ENC','ESC','ESW','EWN','DEU', 'FRA', 'UKI'],
            'TIAM-ECN 1.2': ['WEU'],
            'WITCH 5.1':['AUT','BNL', 'DEU', 'ESP', 'FRA', 'GBR', 'GRC', 'IRL', 'ITA', 'NORTHEU', 'PRT', 'SWE']
        },
        'Eastern Europe': {
            'Euro-Calliope 2.0': ['Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Estonia', 'Hungary', 'Latvia', 'Lithuania', 'Poland', 'Romania', 'Slovakia', 'Slovenia'],
            'IMAGE 3.2': ['CEU'],
            'LIMES 2.38': ['Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Estonia', 'Hungary', 'Latvia', 'Lithuania', 'Poland', 'Romania', 'Slovakia', 'Slovenia'],
            'MEESA v1.2': ['ECE', 'ECS'],
            'MESSAGEix-GLOBIOM 1.2': ['EEU'],
            'OSeMBE v1.0.0': ['BGR', 'CYP', 'CZE', 'EST', 'HRV', 'HUN', 'LTU', 'LVA', 'POL', 'ROU', 'SVK', 'SVN'],
            'PRIMES 2022': ['Eastern Europe (IP)', 'South-East Europe (IP)'],
            'PROMETHEUS 1.2': ['Central Europe'],
            'REMIND 2.1': ['ECE','ECS'],
            'TIAM-ECN 1.2': ['EEU'],
            'WITCH 5.1': ['CZE', 'EASTEU', 'POL', 'ROU']
        }
    },
    'Nine Regions': {
        'United Kingdom and Ireland': {
            'Euro-Calliope 2.0': ['Ireland', 'United Kingdom'],
            'LIMES 2.38': ['Ireland', 'United Kingdom'],
            'MEESA v1.2': ['UKI'],
            'OSeMBE v1.0.0': ['IRL', 'GBR'],
            'PRIMES 2022': ['United Kingdom & Ireland (IP)'],
            'REMIND 2.1': ['UKI'],
            'WITCH 5.1': ['IRL', 'GBR'] 
        },
        'Europe West North': {
            'Euro-Calliope 2.0': ['Austria', 'Belgium', 'Luxembourg', 'Netherlands'],
            'LIMES 2.38': ['Austria', 'Belgium', 'Luxembourg', 'Netherlands'],
            'MEESA v1.2': ['EWN'],
            'OSeMBE v1.0.0': ['AUT', 'BEL', 'LUX', 'NLD'],
            'REMIND 2.1': ['EWN'],
            'WITCH 5.1': ['AUT', 'BNL'] 
        },
        'Europe North Central': {
            'Euro-Calliope 2.0': ['Denmark', 'Finland', 'Sweden'],
            'LIMES 2.38': ['Denmark', 'Finland', 'Sweden'],
            'MEESA v1.2': ['ENC'],
            'OSeMBE v1.0.0': ['DNK', 'FIN', 'SWE'],
            'PRIMES 2022': ['Scandinavia (IP)'],
            'REMIND 2.1': ['ENC'],
            'WITCH 5.1': ['NORTHEU', 'SWE'] 
        },
        'France': {
            'Euro-Calliope 2.0': ['France'],
            'LIMES 2.38': ['France'],
            'MEESA v1.2': ['FRA'],
            'OSeMBE v1.0.0':['FRA'],
            'REMIND 2.1': ['FRA'],
            'WITCH 5.1': ['FRA']
        },
        'Germany': {
            'Euro-Calliope 2.0': ['Germany'],
            'LIMES 2.38': ['Germany'],
            'MEESA v1.2': ['DEU'],
            'OSeMBE v1.0.0':['DEU'],
            'REMIND 2.1': ['DEU'],
            'WITCH 5.1': ['DEU']
        },
        'Europe Central East': {
            'Euro-Calliope 2.0': ['Czech Republic', 'Estonia',  'Latvia', 'Lithuania', 'Poland', 'Slovakia'],
            'LIMES 2.38': ['Czech Republic', 'Estonia',  'Latvia', 'Lithuania', 'Poland', 'Slovakia'],
            'MEESA v1.2': ['ECE'],
            'OSeMBE v1.0.0': ['CZE', 'EST', 'LTU', 'LVA', 'POL', 'SVK'],
            'PRIMES 2022': ['Eastern Europe (IP)'],
            'REMIND 2.1': ['ECE'],
            'WITCH 5.1': ['CZE', 'EASTEU', 'POL']
        },
        'Europe South West': {
            'Euro-Calliope 2.0': ['Portugal', 'Spain'],
            'LIMES 2.38': ['Portugal', 'Spain'],
            'MEESA v1.2': ['ESW'],
            'OSeMBE v1.0.0': ['ESP', 'PRT'],
            'REMIND 2.1': ['ESW'],
            'WITCH 5.1': ['ESP', 'PRT'] 
        },
        'Europe South Central': {
            'Euro-Calliope 2.0': ['Cyprus', 'Greece', 'Italy'],
            'LIMES 2.38': ['Cyprus', 'Greece', 'Italy', 'Malta'],
            'MEESA v1.2': ['ESC'],
            'OSeMBE v1.0.0': ['CYP', 'GRC', 'ITA', 'MLT'],
            'PRIMES 2022': ['Southern Europe (IP)'],
            'REMIND 2.1': ['ESC'],
            'WITCH 5.1': ['GRC','ITA'] 
        },
        'Europe Central South': {
            'Euro-Calliope 2.0': ['Bulgaria', 'Croatia', 'Hungary', 'Romania', 'Slovenia'],
            'LIMES 2.38': ['Bulgaria', 'Croatia', 'Hungary', 'Romania', 'Slovenia'],
            'MEESA v1.2': ['ECS'],
            'OSeMBE v1.0.0': ['BGR', 'HRV', 'HUN', 'ROU', 'SVN'],
            'PRIMES 2022': ['South-East Europe (IP)'],
            'REMIND 2.1': ['ECS'],
            'WITCH 5.1': ['EASTEU', 'ROU']
        }
    },
    'Countries':{
        'Austria': {
            'Euro-Calliope 2.0': ['Austria'],
            'LIMES 2.38': ['Austria'],
            'OSeMBE v1.0.0': ['AUT'],
            'WITCH 5.1': ['AUT']
        },
        'Belgium': {
            'Euro-Calliope 2.0': ['Belgium'],
            'LIMES 2.38': ['Belgium'],
            'OSeMBE v1.0.0':['BEL']
        },
        'Bulgaria': {
            'Euro-Calliope 2.0': ['Bulgaria'],
            'LIMES 2.38': ['Bulgaria'],
            'OSeMBE v1.0.0':['BGR']
        },
        'Croatia': {
            'Euro-Calliope 2.0': ['Croatia'],
            'LIMES 2.38': ['Croatia'],
            'OSeMBE v1.0.0':['HRV']
        },
        'Cyprus': {
            'Euro-Calliope 2.0': ['Cyprus'],
            'LIMES 2.38': ['Cyprus'],
            'OSeMBE v1.0.0':['CYP']
        },
        'Czech Republic': {
            'Euro-Calliope 2.0': ['Czech Republic'],
            'LIMES 2.38': ['Czech Republic'],
            'OSeMBE v1.0.0':['CZE'],
            'WITCH 5.1': ['CZE']
        },
        'Denmark': {
            'Euro-Calliope 2.0': ['Denmark'],
            'LIMES 2.38': ['Denmark'],
            'OSeMBE v1.0.0':['DNK']
        },
        'Estonia': {
            'Euro-Calliope 2.0': ['Estonia'],
            'LIMES 2.38': ['Estonia'],
            'OSeMBE v1.0.0':['EST']
        },
        'Finland': {
            'Euro-Calliope 2.0': ['Finland'],
            'LIMES 2.38': ['Finland'],
            'OSeMBE v1.0.0':['FIN']
        },
        'France': {
            'Euro-Calliope 2.0': ['France'],
            'LIMES 2.38': ['France'],
            'MEESA v1.2': ['FRA'],
            'OSeMBE v1.0.0':['FRA'],
            'REMIND 2.1': ['FRA'],
            'WITCH 5.1': ['FRA']
        },
        'Germany': {
            'Euro-Calliope 2.0': ['Germany'],
            'LIMES 2.38': ['Germany'],
            'MEESA v1.2': ['DEU'],
            'OSeMBE v1.0.0':['DEU'],
            'REMIND 2.1': ['DEU'],
            'WITCH 5.1': ['DEU']
        },
        'Greece': {
            'Euro-Calliope 2.0': ['Greece'],
            'LIMES 2.38': ['Greece'],
            'OSeMBE v1.0.0':['GRC'],
            'WITCH 5.1': ['GRC']
        },
        'Hungary': {
            'Euro-Calliope 2.0': ['Hungary'],
            'LIMES 2.38': ['Hungary'],
            'OSeMBE v1.0.0':['HUN']
        },
        'Ireland': {
            'Euro-Calliope 2.0': ['Ireland'],
            'LIMES 2.38': ['Ireland'],
            'OSeMBE v1.0.0':['IRL'],
            'WITCH 5.1': ['IRL']
        },
        'Italy': {
            'Euro-Calliope 2.0': ['Italy'],
            'LIMES 2.38': ['Italy'],
            'OSeMBE v1.0.0': ['ITA'],
            'WITCH 5.1': ['ITA']
        },
        'Latvia': {
            'Euro-Calliope 2.0': ['Latvia'],
            'LIMES 2.38': ['Latvia'],
            'OSeMBE v1.0.0':['LVA']
        },
        'Lithuania':{
            'Euro-Calliope 2.0': ['Lithuania'],
            'LIMES 2.38': ['Lithuania'],
            'OSeMBE v1.0.0':['LTU']
        } ,
        'Luxembourg': {
            'Euro-Calliope 2.0': ['Luxembourg'],
            'LIMES 2.38': ['Luxembourg'],
            'OSeMBE v1.0.0':['LUX']
            },
        'Malta': {
            'LIMES 2.38': ['Malta'],
            'OSeMBE v1.0.0': ['MLT']
        },
        'Netherlands': {
            'Euro-Calliope 2.0': ['The Netherlands'],
            'LIMES 2.38': ['The Netherlands'],
            'OSeMBE v1.0.0':['NLD']
        },
        'Poland': {
            'Euro-Calliope 2.0': ['Poland'],
            'LIMES 2.38': ['Poland'],
            'OSeMBE v1.0.0':['POL'],
            'WITCH 5.1': ['POL']
        },
        'Portugal': {
            'Euro-Calliope 2.0': ['Portugal'],
            'LIMES 2.38': ['Portugal'],
            'OSeMBE v1.0.0':['PRT'],
            'WITCH 5.1': ['PRT']
        },
        'Romania': {
            'Euro-Calliope 2.0': ['Romania'],
            'LIMES 2.38': ['Romania'],
            'OSeMBE v1.0.0':['ROU'],
            'WITCH 5.1': ['ROU']
        },
        'Slovakia': {
            'Euro-Calliope 2.0': ['Slovakia'],
            'LIMES 2.38': ['Slovakia'],
            'OSeMBE v1.0.0':['SVK']
        },
        'Slovenia': {
            'Euro-Calliope 2.0': ['Slovenia'],
            'LIMES 2.38': ['Slovenia'],
            'OSeMBE v1.0.0':['SVN']
        },
        'Spain': {
            'Euro-Calliope 2.0': ['Spain'],
            'LIMES 2.38':['Spain'],
            'OSeMBE v1.0.0':['ESP'],
            'WITCH 5.1': ['ESP']
        },
        'Sweden': {
            'Euro-Calliope 2.0': ['Sweden'],
            'LIMES 2.38': ['Sweden'],
            'OSeMBE v1.0.0':['SWE'],
            'WITCH 5.1': ['SWE']
        },
        'United Kingdom':{
            'Euro-Calliope 2.0': ['United Kingdom'],
            'LIMES 2.38':['United Kingdom'],
            'OSeMBE v1.0.0':['GBR'],
            'WITCH 5.1': ['GBR']
        }
    }
}

#### Harmonised regions

In [5]:
dic_harm_reg = {
    'Two Regions': {
        'Western Europe': ['Austria', 'Belgium', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Ireland', 'Italy', 'Luxembourg', 'Malta', 'Netherlands', 'Portugal', 'Spain', 'Sweden', 'United Kingdom'],
        'Eastern Europe': ['Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Estonia', 'Hungary', 'Latvia', 'Lithuania', 'Poland', 'Romania', 'Slovakia', 'Slovenia'],
    },
    'Nine Regions': {
        'United Kingdom and Ireland': ['Ireland', 'United Kingdom'],
        'Europe West North': ['Austria', 'Belgium', 'Luxembourg', 'Netherlands'],
        'Europe North Central': ['Denmark', 'Finland', 'Sweden'],
        'France': ['France'],
        'Germany': ['Germany'],
        'Europe Central East': ['Czech Republic', 'Estonia',  'Latvia', 'Lithuania', 'Poland', 'Slovakia'],
        'Europe South West': ['Portugal', 'Spain'],
        'Europe South Central': ['Cyprus', 'Greece', 'Italy', 'Malta'],
        'Europe Central South': ['Bulgaria', 'Croatia', 'Hungary', 'Romania', 'Slovenia']
    },
    'Countries': [
        'Denmark', 'Sweden', 'Finland', 'Estonia', 'Latvia',
        'Ireland', 'United Kingdom', 'Netherlands', 'Poland', 'Lithuania',
        'France', 'Belgium', 'Luxembourg', 'Czech Republic', 'Slovakia', 
        'Spain', 'Germany', 'Austria', 'Hungary', 'Romania',
        'Portugal', 'Italy', 'Greece', 'Slovenia', 'Bulgaria',
        'Malta', 'Cyprus', 'Croatia' 
    ]
}

In [6]:
dic_reg_double_reported = {
    'WITCH': ['AUT','DEU','FRA','GRC','ITA', 'POL'],
    'REMIND': ['DEU', 'FRA', 'UKI', 'ENC', 'ECE', 'ESC', 'ECS'],
    'MEESA': ['ENC', 'UKI', 'ESC', 'ECS', 'ECE']
}

### Get data from IIASA ScenarioExplorer

In [7]:
conn = pyam.iiasa.Connection('ecemf_internal')
df_common_variables = conn.query(
    model=wp1_models,
    variable=common_variables,
    scenario= interest_scenarios
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.iiasa - INFO: You are connected to the IXSE_ECEMF_INTERNAL scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ece.iiasa.ac.at/ecemf-internal/#/about
pyam.iiasa - INFO: You are connected as user `haukehenke`


### Data adjustments

#### Adjust solar power reporting

In [8]:
for model in [m for m in wp1_models if m not in df_common_variables.filter(variable='Secondary Energy|Electricity|Solar').model]:
    df = df_common_variables.filter(variable='Secondary Energy|Electricity|Solar|PV', model=model).data
    df['variable'] = 'Secondary Energy|Electricity|Solar'
    df_common_variables = df_common_variables.append(df)

**Euro-Calliope consider for solar only PV, not CSP**

In [9]:
df = df_common_variables.filter(variable='Secondary Energy|Electricity|Solar', model='Euro-Calliope 2.0').data
df['variable'] = 'Secondary Energy|Electricity|Solar|PV'
df_common_variables = df_common_variables.append(df)

**MEESA reports Solar PV Utility and Rooftop**

In [10]:
df = df_common_variables.filter(model='MEESA v1.2', variable='Capacity|Electricity|Solar|PV|*').data.groupby(by=['model', 'scenario', 'region', 'unit', 'year'], as_index=False).sum()
df['variable'] = 'Capacity|Electricity|Solar|PV'
df_common_variables = df_common_variables.append(df)

In [11]:
for model in [m for m in wp1_models if m not in df_common_variables.filter(variable='Capital Cost|Electricity|Solar|PV').model]:
    df = df_common_variables.filter(variable='Capital Cost|Electricity|Solar|PV|Utility', model=model).data
    df['variable'] = 'Capital Cost|Electricity|Solar|PV'
    df_common_variables = df_common_variables.append(df)

pyam.core - WARNING: Filtered IamDataFrame is empty!
pyam.utils - WARNING: Formatted data is empty!
pyam.core - WARNING: Filtered IamDataFrame is empty!
pyam.utils - WARNING: Formatted data is empty!


#### Adjust Hydrogen reporting
Euro-Calliope reports only `Secondary Energy|Hydrogen` and not  `Secondary Energy|Hydrogen|Electricity`, but all hydrogen in Euro-Calliope is generated via electrolysis. Hence, in the case of Euro-Calliope the former can be used when comparing the latter across models.

In [12]:
df = df_common_variables.filter(variable='Secondary Energy|Hydrogen', model='Euro-Calliope 2.0').data
df['variable'] = 'Secondary Energy|Hydrogen|Electricity'
df_common_variables = df_common_variables.append(df)

### Functions

In [13]:
def model_regions_str(wanted_models: Dict)->List:
    mod_reg_str = []
    mod_reg__str_dics = {}
    for m in wanted_models:
        mod_reg__str_dics[m] = []
        if m == 'PRIMES 2022':
            for r in wanted_models[m]:
                mod_reg_str.append(r)
                mod_reg__str_dics[m].append(r)
        elif m == 'OSeMBE v1.0.0':
            for r in wanted_models[m]:
                mod_reg_str.append('OSeMBE v1.0|'+r)
                mod_reg__str_dics[m].append('OSeMBE v1.0|'+r)
        elif m == 'LIMES 2.38':
            for r in wanted_models[m]:
                mod_reg_str.append(r)
                mod_reg__str_dics[m].append(r)
        else:
            for r in wanted_models[m]:
                mod_reg_str.append(m+'|'+r)
                mod_reg__str_dics[m].append(m+'|'+r)
    return mod_reg_str, mod_reg__str_dics

In [14]:
def df_plot(data: pyam.IamDataFrame, models: List, years: List, variable: str, region: str, regions: List):
    df = data.filter(model=models, scenario='DIAG-C400-lin', year=years, variable=variable, region=regions).data
    df['agg_region'] = region
    return df

In [15]:
def get_model_colors(models: List):
    color_list = []
    model_colors = {
        'Euro-Calliope 2.0': "darkgrey",
        'LIMES 2.38': "lightgreen",
        'OSeMBE v1.0.0': "firebrick",
        'WITCH 5.1': "forestgreen",
        'MEESA v1.2': "magenta",
        'REMIND 2.1': "tomato",
        'IMAGE 3.2': "turquoise",
        'MESSAGEix-GLOBIOM 1.2': "purple",
        'PRIMES 2022': "goldenrod",
        'PROMETHEUS 1.2': "yellow",
        'TIAM-ECN 1.2': "steelblue"
    }
    for m in models:
        color_list.append(model_colors[m])
    return color_list

In [16]:
def filter_double_countries(lst_region: List, model: str, regions: List):
    expr_model_out = '^((?!'+model+').)*$'
    mask_rem_model = re.compile(expr_model_out)
    lst_reg_not_aff = list(filter(mask_rem_model.match, lst_region))
    expr_model_only = '.*('+model+')'
    mask_model_only = re.compile(expr_model_only)
    lst_model_reg = list(filter(mask_model_only.match, lst_region))
    expr_excl_reg = '^((?!'
    i = 1
    l = len(regions)
    for r in regions:
        if i < l:
            expr_excl_reg = expr_excl_reg + '(' + r + ')|'
        else:
            expr_excl_reg = expr_excl_reg + '(' + r + ')).)*$'
        i+=1
    mask_excl_reg = re.compile(expr_excl_reg)
    lst_model_reg_sel = list(filter(mask_excl_reg.match, lst_model_reg))
    return lst_reg_not_aff + lst_model_reg_sel

The below function is currently not used. For variables that should not be summed when working with aggregated regions, the function check the values reported for model and only provides a value for the plot of the values reported across regions are equal. 

In [ ]:
def create_cost_df(df: pd.DataFrame):
    df_cost_agg_reg = pd.DataFrame()
    for ar in df['agg_region'].unique():
        df_ag = df[df['agg_region']==ar]
        for m in df_ag['model'].unique():
            print(m)
            df_ag_m = df_ag[df_ag['model']==m]
            for yr in df_ag_m['year'].unique():
                df_ag_m_yr = df_ag_m[df_ag_m['year']==yr]
                if m == 'PRIMES 2022':
                    print(yr, df_ag_m_yr.head())
                if df_ag_m_yr['value'].nunique()<2:
                    df_new_rows = pd.DataFrame([[m, yr, ar, df_ag_m_yr['value'].iloc[0]]], columns=['model', 'year', 'agg_region', 'value'])
                    df_cost_agg_reg = pd.concat([df_cost_agg_reg, df_new_rows])
    return df_cost_agg_reg

#### Function to get variable stats

In [17]:
def get_var_stats(df: pd.DataFrame):
    median = df['value'].median()
    stdv = df['value'].std(ddof=0) # ddof=0 makes the function use n instead of n-1 in the stdv calculation
    max = df.loc[df['value'].idxmax()]['value']
    min = df.loc[df['value'].idxmin()]['value']
    return median, stdv, max, min

### Create plots and tables

In [99]:
table_dic = {}
for agg_reg in dic_model_reg:
    for r in dic_model_reg[agg_reg]:
        table_dic[r] = pd.DataFrame(index=wp1_models)

# sel_variables = ['Secondary Energy|Electricity|Solar|PV', 'Capital Cost|Electricity|Solar|PV']
# for c in sel_variables:

# for c in common_variables:
#     print(c)
c = 'Secondary Energy|Hydrogen'
v_path_str = c.replace('|', '_')
for d in dic_model_reg:
    # d='Nine Regions'

    n_plots = len(dic_model_reg[d].keys())
    if n_plots==2:
        n =2
    else:
        n =  round(n_plots**(1/2))

    df = pd.DataFrame()
    yrs = {'2050': [2050], '10 year steps': [2010,2020,2030,2040,2050]}

    for r in dic_model_reg[d]:
        modls = list(dic_model_reg[d][r])
        number_of_model_regions = []
        for m in list(dic_model_reg[d][r].keys()):
            number_of_model_regions.append(len(dic_model_reg[d][r][m]))
        max_n_regs = max(number_of_model_regions)
        list_model_reg_str, dic_model_reg_str = model_regions_str(dic_model_reg[d][r])

        for m in dic_reg_double_reported:
            list_model_reg_str = filter_double_countries(list_model_reg_str, m, dic_reg_double_reported[m])

        df = pd.concat([df, df_plot(df_common_variables, modls, yrs['10 year steps'], c, r, list_model_reg_str)])
        
    if not df.empty:
        if any(t in c for t in ('Cost', 'Life', 'Efficiency')):
            df_plots = df.groupby(['model', 'year', 'agg_region']).mean()
        else:  
            df_plots = df.groupby(['model', 'year', 'agg_region']).sum()
        df_plots = df_plots.reset_index()
        df_plots.loc[df_plots['model'].isin(model_type['IAM']), 'type'] = 'IAM'
        df_plots.loc[df_plots['model'].isin(model_type['ESM']), 'type'] = 'ESM'
        plot_colors = get_model_colors(list(df_plots['model'].unique()))

        agg_region = []
        for r in dic_harm_reg[d]:
            if r in df_plots['agg_region'].unique():
                agg_region.append(r)

        fig = px.line(
            df_plots, 
            x='year', y='value', 
            facet_col='agg_region', facet_col_wrap=n, facet_col_spacing=0.05,
            title='<b>'+c+'</b>'+' for Europe aggregated in '+d,
            color='model',
            line_dash='type',
            color_discrete_sequence=plot_colors,
            labels={'model': 'Model', 'year': "Year", 'license': 'License:'},
            category_orders= {"agg_region": agg_region},
            height=600, width=1067
            )
        
        if d=='Countries':
            fig.update_yaxes(matches=None, showticklabels=True)
        for axis in fig.layout:
            if type(fig.layout[axis]) == go.layout.YAxis:
                fig.layout[axis].title.text = ''

        fig.update_layout(
            # keep the original annotations and add a list of new annotations:
            annotations = list(fig.layout.annotations) + 
            [go.layout.Annotation(
                    x=-0.07,
                    y=0.5,
                    font=dict(
                        size=14
                    ),
                    showarrow=False,
                    text=df['unit'].unique()[0],
                    textangle=-90,
                    xref="paper",
                    yref="paper"
                )
            ]
        )
        fig.update_layout(legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.25,
            xanchor="center",
            x=0.5
        ))
        fig.update_layout(
            font = dict(
                size=16
            )
        )
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


        if n_plots>2:
            row = math.ceil(n_plots**(1/2))
        else:
            row = 1
        col = 1
        for p in agg_region:
            
            if not df_plots[(df_plots['year']==2050)&(df_plots['agg_region']==p)].empty:
                median, standev, maxi, mini = get_var_stats(df_plots[(df_plots['year']==2050)&(df_plots['agg_region']==p)])
                
                if (median-standev-mini) >0:
                    low = median-standev-mini
                else:
                    low = 0

                if ((median-standev) > mini)&((median+standev) < maxi):
                    medium = 2*standev
                elif (median+standev) < maxi:
                    medium = (median-mini)+standev
                elif (median-standev) > mini:
                    medium = (maxi-median)+standev
                else:
                    medium = maxi-mini
                
                if maxi-standev > median:
                    high = maxi-median-standev
                else:
                    high = 0
                y = [low, medium, high]

                base_mini = mini
                if median-standev>mini:
                    base_medium = median-standev
                else:
                    base_medium = mini
                if median+standev<maxi:
                    base_high = median+standev
                else:
                    base_high = maxi    
                base = [base_mini, base_medium, base_high]
                fig.add_trace( 
                    go.Bar(
                        x = [2050, 2050, 2050],
                        y = y,
                        base = base,
                        marker_color= ['blue', 'yellow', 'red'],
                        opacity=0.5,
                        name='Level'
                    ),
                    row=row,
                    col=col
                )
                if col < n:
                    col += 1
                else:
                    col = 1
                    row -= 1

            new_col = df_plots[(df_plots['year']==2050)&(df_plots['agg_region']==p)]
            new_col = new_col.set_index('model')
            new_col = new_col.rename(columns={'value': c})
            if not new_col.empty:
                if not c in table_dic[p]:
                    table_dic[p] = pd.concat([table_dic[p], new_col[c]], axis=1)
            # levels = ['low', 'medium', 'high']
            # conditions = [
            #     (new_col['value']<(median-standev)),
            #     (new_col['value']>=(median-standev))&((new_col['value']<(median+standev))),
            #     (new_col['value']>=(median+standev))
            # ]

            # new_col[c] = np.select(conditions, levels)

    fig.show()

        # path_html = os.path.join(path_dic, v_path_str+'-'+d+'.html')
        # fig.write_html(path_html)
        # path_png = os.path.join(path_dic,v_path_str+'-'+d+'.png')
        # fig.write_image(path_png) #, scale=2)


pyam.core - WARNING: Filtered IamDataFrame is empty!


In [70]:
# Function to highlight cells in dataframe column that are either within +-standard deviations from the median, higher, or lower.
def highlight(x):
    stdv = x.std(ddof=0)
    med = x.median()
    return np.where(x > (med+stdv), f"background-color: red;", np.where(x<(med-stdv), f"background-color: blue;", f"background-color: yellow;"))

In [71]:
# Color coding dataframe by column and writing out as excel file
for r in table_dic:
    if not table_dic[r].empty:
        s = table_dic[r].style.apply(highlight)
        s.to_excel(os.path.join(path_dic, 'variable-model-comparison-'+r+'.xlsx')) 

Region:  Western Europe                        Capacity|Electricity|Nuclear  \
IMAGE 3.2                                 15.027340   
MESSAGEix-GLOBIOM 1.2                     72.865649   
PROMETHEUS 1.2                                  NaN   
REMIND 2.1                                28.917052   
TIAM-ECN 1.2                              63.350000   
WITCH 5.1                                 51.721407   
Euro-Calliope 2.0                         30.900727   
LIMES 2.38                                 1.749200   
MEESA v1.2                               110.678000   
OSeMBE v1.0.0                             19.060000   
PRIMES 2022                               55.375760   

                       Capacity|Electricity|Solar|CSP  \
IMAGE 3.2                                    2.316860   
MESSAGEix-GLOBIOM 1.2                        0.294118   
PROMETHEUS 1.2                                    NaN   
REMIND 2.1                                   0.487038   
TIAM-ECN 1.2                  

In [ ]:
test_line = pd.DataFrame([
    [2010, 5, 'a'], [2020, 6, 'a'], [2030, 8, 'a'], [2040, 11, 'a'], [2050, 15, 'a'],
    [2010, 6, 'b'], [2020, 7, 'b'], [2030, 9, 'b'], [2040, 12, 'b'], [2050, 16, 'b'],
    [2010, 4, 'c'], [2020, 5, 'c'], [2030, 7, 'c'], [2040, 10, 'c'], [2050, 14, 'c']
    ],
    columns=['year', 'value', 'series'])

max = test_line.loc[test_line[test_line['year']==2050]['value'].idxmax()]['value']
min = test_line.loc[test_line[test_line['year']==2050]['value'].idxmin()]['value']
third = (max - min)/3

df_bar_2050 = pd.DataFrame(columns=['year', 'value', 'base', 'level'])

i = min 
for j in ['low', 'medium', 'high']:
    row = pd.DataFrame([[2050, third, i, j]], columns=['year', 'value', 'base', 'level'])
    df_bar_2050 = pd.concat([df_bar_2050, row])
    i += third

fig_line = px.line(test_line, x='year', y='value', color='series')

fig_line.add_bar(
    x=df_bar_2050.year,
    y=df_bar_2050.value,
    base=df_bar_2050.base,
    marker_color= ['blue', 'yellow', 'red'],
    opacity=0.5)

fig_line.show()

In [ ]:
def level_str(x, stdv, med):
    if x > (med+stdv):
        return 'high'
    elif x < (med-stdv):
        return 'low'
    return 'medium'
def highlight_high(x, color):
    print(x)
    stdv = 4 #x.std(ddof=0)
    med = 2 #x.median()
    # print(stdv, med)
    x.format(level_str, stdv, med)
    print(x)
    return x #np.where(x > (med+stdv), f"background-color: {color};", None)
# def highlight_low(x, color):
#     stdv = x.std(ddof=0)
#     med = x.median()
#     print(stdv, med)
#     return np.where(x < (med-stdv), f"background-color: {color};", None)
df = pd.DataFrame(np.random.randn(5, 2), columns=["A", "B"])
s = df.style.pipe(highlight_high, color='yellow')
# s = s.style.apply(highlight_high, color='red')  
s
# df.style.apply(highlight_high, color='red', axis=1)  
# df.style.apply(highlight_max, color='green', axis=None)  

### Plot countries of selected aggregate region

In [ ]:
def plot_cs_of_aggregate_reg(path: str, variable: str, aggregation: str, reg: str, harm_reg: Dict, model_reg: Dict, data):
    v_path_str = variable.replace('|', '_')
    n_plots = len(harm_reg[aggregation][reg])
    if n_plots==2:
            n_col =2
    else:
        n_col =  round(n_plots**(1/2))
    
    df = pd.DataFrame()
    yrs = [2010,2020,2030,2040,2050]
    for r in harm_reg[aggregation][reg]:
        models = list(model_reg['Countries'][r])
        lst_model_reg_str, dic_mod_reg_str = model_regions_str(dic_model_reg['Countries'][r])
        for m in dic_reg_double_reported:
            lst_model_reg_str = filter_double_countries(lst_model_reg_str, m, dic_reg_double_reported[m])
        df = pd.concat([df, df_plot(data, models, yrs, variable, r, lst_model_reg_str)])
    if not df.empty:
        plot_colors = get_model_colors(list(df['model'].unique()))
        fig = px.line(
            df, 
            x='year', y='value', 
            facet_col='agg_region', facet_col_wrap=n_col, facet_col_spacing=0.05,
            title='<b>'+variable+'</b>'+' for '+ reg +' in country resolution',
            color='model',
            color_discrete_sequence=plot_colors,
            labels={'model': 'Model:', 'year': "Year"},
            height=600, width=1067
            )
        fig.update_yaxes(matches=None, showticklabels=True)
        for axis in fig.layout:
            if type(fig.layout[axis]) == go.layout.YAxis:
                fig.layout[axis].title.text = ''

        fig.update_layout(
            # keep the original annotations and add a list of new annotations:
            annotations = list(fig.layout.annotations) + 
            [go.layout.Annotation(
                    x=-0.07,
                    y=0.5,
                    font=dict(
                        size=14
                    ),
                    showarrow=False,
                    text=df['unit'].unique()[0],
                    textangle=-90,
                    xref="paper",
                    yref="paper"
                )
            ]
        )
        fig.update_layout(legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.25,
            xanchor="right",
            x=0.99
        ))
        fig.update_layout(
            font = dict(
                size=16
            )
        )           
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

        path_html = os.path.join(path, v_path_str+'-'+reg+'.html')
        fig.write_html(path_html)
        path_png = os.path.join(path, v_path_str+'-'+reg+'.png')
        fig.write_image(path_png)
        # fig.show()
    return df

In [ ]:
plot_cs_of_aggregate_reg(path_dic, "Secondary Energy|Electricity|Solar", 'Nine Regions', 'Europe Central East', dic_harm_reg, dic_model_reg, df_common_variables)

## Create plots of vRE shares

In [ ]:
def df_vre_plot(data: pd.DataFrame, models: List, years: List, region: str, regions: List):
    df = data[
    (data['region'].isin(regions)) &
    (data['model'].isin(models)) &
    (data['year'].isin(years)) & 
    (data['scenario'].isin(['DIAG-C400-lin']))
    ]
    df['agg_region'] = region
    return df

In [ ]:
df_vre = df_common_variables.filter(variable=['Secondary Energy|Electricity|Solar', 'Secondary Energy|Electricity|Wind']).data.groupby(['year', 'region', 'model']).sum()
df_fee = df_common_variables.filter(variable=['Secondary Energy|Electricity']).data
df_vre = df_vre.reset_index()
df_vre = df_vre.merge(df_fee, how='left', on=['year', 'region', 'model'], suffixes=['_vre', '_fee'])

In [ ]:
c = 'Share|variable Renewable Energies'
v_path_str = c.replace('|', '_')
for d in dic_model_reg:
    # d='Nine Regions'
    n_plots = len(dic_model_reg[d].keys())
    if n_plots==2:
        n =2
    else:
        n =  round(n_plots**(1/2)) # math.ceil((len(dic_model_reg[d].keys()))**(1/2))

    df = pd.DataFrame()
    yrs = {'2050': [2050], '10 year steps': [2010,2020,2030,2040,2050]}

    # if d=='Countries':
    for r in dic_model_reg[d]:
        modls = list(dic_model_reg[d][r])
        number_of_model_regions = []
        for m in list(dic_model_reg[d][r].keys()):
            number_of_model_regions.append(len(dic_model_reg[d][r][m]))
        max_n_regs = max(number_of_model_regions)
        list_model_reg_str, dic_model_reg_str = model_regions_str(dic_model_reg[d][r])

        for m in dic_reg_double_reported:
            list_model_reg_str = filter_double_countries(list_model_reg_str, m, dic_reg_double_reported[m])

        df = pd.concat([df, df_vre_plot(df_vre, modls, yrs['10 year steps'], r, list_model_reg_str)])
        
    if not df.empty:
        df_plots = df.groupby(['model', 'year', 'agg_region']).sum()
        df_plots = df_plots.reset_index()
        df_plots['share vre'] = (df_plots['value_vre'] / df_plots['value_fee']) * 100
        df_plots.loc[df_plots['model'].isin(model_type['IAM']), 'type'] = 'IAM'
        df_plots.loc[df_plots['model'].isin(model_type['ESM']), 'type'] = 'ESM'
        plot_colors = get_model_colors(list(df_plots['model'].unique()))

        agg_region = []
        for r in dic_harm_reg[d]:
            if r in df_plots['agg_region'].unique():
                agg_region.append(r)
                
        fig = px.line(
            df_plots, 
            x='year', y='share vre', 
            facet_col='agg_region', facet_col_wrap=n, facet_col_spacing=0.05,
            title='<b>'+c+'</b>'+' for Europe aggregated in '+d,
            color='model',
            line_dash='type',
            color_discrete_sequence=plot_colors,
            labels={'model': 'Model', 'year': "Year", 'license': 'License:'},
            category_orders={"agg_region": agg_region},
            height=600, width=1067
            )
        
        if d=='Countries':
            fig.update_yaxes(matches=None, showticklabels=True)
        for axis in fig.layout:
            if type(fig.layout[axis]) == go.layout.YAxis:
                fig.layout[axis].title.text = ''

        fig.update_layout(
            # keep the original annotations and add a list of new annotations:
            annotations = list(fig.layout.annotations) + 
            [go.layout.Annotation(
                    x=-0.07,
                    y=0.5,
                    font=dict(
                        size=18
                    ),
                    showarrow=False,
                    text='%',
                    textangle=-90,
                    xref="paper",
                    yref="paper"
                )
            ]
        )
        fig.update_layout(legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.25,
            xanchor="center",
            x=0.5
        ))
        fig.update_layout(
            font = dict(
                size=16
            )
        )
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

        fig.show()

        path_html = os.path.join(path_dic, v_path_str+'-'+d+'.html')
        fig.write_html(path_html)
        path_png = os.path.join(path_dic, v_path_str+'-'+d+'.png')
        fig.write_image(path_png)

## Create plots of Utilisation

In [ ]:
def get_uti_vars(data, se_var: str, capa_var: str)-> pd.DataFrame:
    df_uti = data.filter(variable=[se_var]) 
    df_uti = df_uti.convert_unit('EJ/yr', to='GWh/yr')
    df_uti = df_uti.data.groupby(['year', 'region', 'model']).sum()
    df_capa = df_common_variables.filter(variable=[capa_var]).data
    df_uti = df_uti.reset_index()
    return df_uti.merge(df_capa, how='left', on=['year', 'region', 'model'], suffixes=['_se', '_capa'])

In [ ]:
def df_uti_plot(data: pd.DataFrame, models: List, years: List, region: str, regions: List):
    df = data[
    (data['region'].isin(regions)) &
    (data['model'].isin(models)) &
    (data['year'].isin(years)) & 
    (data['scenario'].isin(['DIAG-C400-lin']))
    ]
    df['agg_region'] = region
    return df

In [ ]:
for v in ['Solar|PV', 'Wind|Offshore', 'Wind|Onshore']:
    c = 'Capacity Factor|' + v
    v_path_str = c.replace('|', '_')

    se_var = 'Secondary Energy|Electricity|' + v
    capa_var = 'Capacity|Electricity|' + v

    df_uti_vars = get_uti_vars(df_common_variables, se_var, capa_var)

    for d in dic_model_reg:
        # d='Nine Regions'
        n_plots = len(dic_model_reg[d].keys())
        if n_plots==2:
            n =2
        else:
            n =  round(n_plots**(1/2)) # math.ceil((len(dic_model_reg[d].keys()))**(1/2))

        df = pd.DataFrame()
        yrs = {'2050': [2050], '10 year steps': [2010,2020,2030,2040,2050]}

        # if d=='Countries':
        for r in dic_model_reg[d]:
            modls = list(dic_model_reg[d][r])
            number_of_model_regions = []
            for m in list(dic_model_reg[d][r].keys()):
                number_of_model_regions.append(len(dic_model_reg[d][r][m]))
            max_n_regs = max(number_of_model_regions)
            list_model_reg_str, dic_model_reg_str = model_regions_str(dic_model_reg[d][r])

            for m in dic_reg_double_reported:
                list_model_reg_str = filter_double_countries(list_model_reg_str, m, dic_reg_double_reported[m])

            df = pd.concat([df, df_uti_plot(df_uti_vars, modls, yrs['10 year steps'], r, list_model_reg_str)])
            
        if not df.empty:
            df_plots = df.groupby(['model', 'year', 'agg_region']).sum()
            df_plots = df_plots.reset_index()
            df_plots['capacity factor'] = (df_plots['value_se'] / (df_plots['value_capa'] * 8760)) * 100
            df_plots.loc[df_plots['model'].isin(model_type['IAM']), 'type'] = 'IAM'
            df_plots.loc[df_plots['model'].isin(model_type['ESM']), 'type'] = 'ESM'
            plot_colors = get_model_colors(list(df_plots['model'].unique()))

            agg_region = []
            for r in dic_harm_reg[d]:
                if r in df_plots['agg_region'].unique():
                    agg_region.append(r)
                    
            fig = px.line(
                df_plots, 
                x='year', y='capacity factor', 
                facet_col='agg_region', facet_col_wrap=n, facet_col_spacing=0.05,
                title='<b>'+c+'</b>'+' for Europe aggregated in '+d,
                color='model',
                line_dash='type',
                color_discrete_sequence=plot_colors,
                labels={'model': 'Model', 'year': "Year", 'license': 'License:'},
                category_orders={"agg_region": agg_region},
                height=600, width=1067
                )
            
            if d=='Countries':
                fig.update_yaxes(matches=None, showticklabels=True)
            for axis in fig.layout:
                if type(fig.layout[axis]) == go.layout.YAxis:
                    fig.layout[axis].title.text = ''

            fig.update_layout(
                # keep the original annotations and add a list of new annotations:
                annotations = list(fig.layout.annotations) + 
                [go.layout.Annotation(
                        x=-0.07,
                        y=0.5,
                        font=dict(
                            size=18
                        ),
                        showarrow=False,
                        text='%',
                        textangle=-90,
                        xref="paper",
                        yref="paper"
                    )
                ]
            )
            fig.update_layout(legend=dict(
                orientation="h",
                yanchor="top",
                y=-0.25,
                xanchor="center",
                x=0.5
            ))
            fig.update_layout(
                font = dict(
                    size=16
                )
            )
            fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

            # fig.show()

            path_html = os.path.join(path_dic, v_path_str+'-'+d+'.html')
            fig.write_html(path_html)
            path_png = os.path.join(path_dic, v_path_str+'-'+d+'.png')
            fig.write_image(path_png)